<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Talleres/Calculo_de_THD_y_Factor_de_Potencia_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar dependencias

In [1]:
!pip install -q streamlit numpy scipy pandas matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


Crear el script am_thd_dashboard.py

In [2]:
%%bash
cat > am_thd_dashboard.py << 'EOF'
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Funciones auxiliares ---
def compute_thd(signal, fs, f0, nharmonics=10):
    N = len(signal)
    Y = np.fft.fft(signal) / N
    freqs = np.fft.fftfreq(N, 1/fs)
    amps = {}
    for n in range(1, nharmonics+1):
        idx = np.argmin(np.abs(freqs - n*f0))
        amps[n] = 2 * np.abs(Y[idx])
    A1 = amps[1]
    harm_sq = sum(amps[n]**2 for n in range(2, nharmonics+1))
    THD = np.sqrt(harm_sq) / A1
    PF_distortion = 1 / np.sqrt(1 + THD**2)
    return THD, PF_distortion, amps

def plot_time(i_t, t, title):
    fig, ax = plt.subplots(figsize=(8,2))
    ax.plot(t, i_t, label='i(t)')
    ax.set(xlabel='t [s]', ylabel='i(t)', title=title)
    ax.grid(True)
    st.pyplot(fig)

def plot_spectrum(amps1, amps2, title, nharmonics):
    n = np.arange(1, nharmonics+1)
    A1 = [amps1[k] for k in n]
    A2 = [amps2[k] for k in n]
    fig, ax = plt.subplots(figsize=(8,3))
    ax.stem(n, A1,            basefmt=" ", label='Resistiva')
    ax.stem(n, A2, linefmt='C1-', markerfmt='C1o', basefmt=" ", label='RC seleccionada')
    ax.set(xlabel='Armónico n', ylabel='Amplitud', title=title)
    ax.legend(); ax.grid(True)
    st.pyplot(fig)

# --- App config ---
st.set_page_config(page_title="THD & PF – Carga RC", page_icon="⚡️", layout="wide")
st.title("Cálculo de THD y Factor de Potencia")

# --- Sidebar inputs ---
st.sidebar.header("Parámetros de la fuente")
f0  = st.sidebar.number_input("Frecuencia fundamental f0 (Hz)",    min_value=1.0,   value=50.0,    step=1.0)
Np  = st.sidebar.number_input("No. de períodos",                   min_value=1,     value=5,       step=1)
spp = st.sidebar.number_input("Muestras por período",              min_value=100,   value=2048,    step=128)
nh  = st.sidebar.number_input("Número de armónicos a considerar",  min_value=1,     value=10,      step=1)

st.sidebar.header("Parámetros de carga RC")
R  = st.sidebar.number_input("Resistencia R (Ω)",                  min_value=1.0,   value=100.0,   step=1.0)
C_mf = st.sidebar.text_input("Capacitancias C (µF), separadas por coma", "10,100,1000")
# convertimos a Faradios
C_list = []
for c in C_mf.split(','):
    try:
        C_list.append(float(c.strip())*1e-6)
    except:
        pass
sel_C = st.sidebar.selectbox("Selecciona C (µF) para comparar espectro",
                             options=[c*1e6 for c in C_list],
                             format_func=lambda x: f"{x:.1f} µF")
C_sel = sel_C * 1e-6

# --- Time vector ---
w0 = 2 * np.pi * f0
T0 = 1 / f0
fs = spp * f0
t  = np.linspace(0, Np * T0, int(Np*spp), endpoint=False)

# --- Señal rectificada resistiva ---
i_rect = np.abs(np.sin(w0 * t))

st.header("1) Señal rectificada (carga resistiva)")
plot_time(i_rect, t, "i_rect(t) – |sin(ω₀t)|")

# --- Cálculo THD / PF resistiva ---
THD_res, PF_res, amps_res = compute_thd(i_rect, fs, f0, nharmonics=nh)
st.markdown(f"**Resistiva → THD = {THD_res:.4f}, PF (distortion) = {PF_res:.4f}**")

# --- Cálculo para cada C en lista ---
results = []
for C in C_list:
    H = lambda n: 1/np.sqrt(1 + (n * w0 * R * C)**2)
    amps_rc = {n: amps_res[n] * H(n) for n in amps_res}
    A1_rc = amps_rc[1]
    harm_sq_rc = sum(amps_rc[n]**2 for n in range(2, nh+1))
    THD_rc = np.sqrt(harm_sq_rc) / A1_rc
    PF_rc  = 1 / np.sqrt(1 + THD_rc**2)
    results.append({
        'C [µF]': C*1e6,
        'THD':     THD_rc,
        'PF_dist': PF_rc
    })

df = pd.DataFrame(results)
st.header("2) THD y PF para diferentes C")
st.dataframe(df.style.format({'C [µF]':'{:.1f}','THD':'{:.4f}','PF_dist':'{:.4f}'}), height=200)

# --- Espectro comparativo ---
st.header(f"3) Espectro de armónicos (hasta n={nh})")
# espectros
N = len(i_rect)
Y = np.fft.fft(i_rect)/N
freqs = np.fft.fftfreq(N, 1/fs)
# amps resistiva
amps_rect = {n: 2*abs(Y[np.argmin(abs(freqs - n*f0))]) for n in range(1, nh+1)}
# amps RC seleccionada
Hsel = lambda n: 1/np.sqrt(1 + (n * w0 * R * C_sel)**2)
amps_filt = {n: amps_rect[n]*Hsel(n) for n in amps_rect}
plot_spectrum(amps_rect, amps_filt, f"Armónicos vs RC {sel_C:.1f} µF", nharmonics=nh)
EOF


 Arranca Streamlit y el túnel

In [9]:
!pkill -f streamlit || true
!tail -n 30 /content/logs.txt || echo "No hay logs aún"


^C
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [12]:
%%bash
# Descarga y prepara Cloudflared
wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
chmod +x cloudflared-linux-amd64
mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Arranca tu dashboard (usa el nombre correcto)
nohup streamlit run am_thd_dashboard.py --server.address=0.0.0.0 --server.port=8501 &> /content/logs.txt &

# Inicia el túnel
nohup cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &


Extrae tu URL pública

In [13]:
import time, re, os

# Espera a que Cloudflared emita la URL
time.sleep(5)

url = None
with open('/content/cloudflared.log') as f:
    for line in f:
        # Busca cualquier enlace .trycloudflare.com
        m = re.search(r'https://[\w\.-]+\.trycloudflare\.com', line)
        if m:
            url = m.group(0)
            break

if url:
    print(f"✅ Tu aplicación está disponible en:\n{url}")
else:
    print("⚠️ No encontré la URL. Revisa cloudflared.log:\n")
    print(open('/content/cloudflared.log').read())

# Opción para detener el servidor cuando termines
res = input("Escribe '1' para detener Streamlit: ")
if res.strip() == "1":
    os.system("pkill streamlit")
    print("🔴 Streamlit detenido.")


✅ Tu aplicación está disponible en:
https://pf-slowly-hundred-elements.trycloudflare.com
Escribe '1' para detener Streamlit: 1
🔴 Streamlit detenido.
